# 문제 1 Tokenizer 생성하기
**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

In [ ]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    '''
    문제 1-1.
    '''
    import re
    for s in sequences:
      s = s.lower().strip()                                                     # 소문자 변환과 양끝 공백제거
      s = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ', s)# 특수문자 제거, (')가 들어간 불용어는 따로 분리하기 위해 공백으로 교체
      s = re.sub(r'[" "]+', " ", s)                                             # 여러개의 공백은 한개의 공백으로, 특수문자가 공백으로 바뀌면서 생기는 2개이상 공백을 1개로 교체
      s = s.split() 
      result.append(s)
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    '''
    문제 1-2.
    '''
    tokens = self.preprocessing(sequences)                                      # 토큰화 수행
    for s in tokens:                                                            # 토큰내 문장
      for w in s:                                                               # 문장내 단어
        if w not in self.word_dict.keys():                                      # 단어사전에 없는 단어 일 경우
          self.word_dict[w] = len(self.word_dict)                               # 단어사전에 추가
    self.fit_checker = True
  
  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      '''
      문제 1-3.
      '''
      for s in tokens:                                                          # 토큰내 문장
        indexes = []                                                            # 문장별 인덱스 리스트
        for w in s:                                                             # 문장내 단어
          if w in self.word_dict.keys():                                        # 단어사전에 단어가 있으면
            idx = self.word_dict[w]                                             # 해당 단어의 인덱스
          else:                                                                 # 없으면
            idx = self.word_dict['oov']                                         # oov의 인덱스
          indexes.append(idx)                                                   # 문장별 인덱스 완성
        result.append(indexes)
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [ ]:
# Tokenizer.preprocessing 테스트
tokenizer = Tokenizer()
sequence = ["I'm going to school.", 'I LIKE pizza! ']
print(tokenizer.preprocessing(sequence))

[['i', 'm', 'going', 'to', 'school'], ['i', 'like', 'pizza']]


In [ ]:
# Tokenizer.fit 테스트
tokenizer.fit(sequence)
print(tokenizer.word_dict)

{'oov': 0, 'i': 1, 'm': 2, 'going': 3, 'to': 4, 'school': 5, 'like': 6, 'pizza': 7}


In [ ]:
# Tokenizer.transform 테스트
print(tokenizer.transform(sequence))

[[1, 2, 3, 4, 5], [1, 6, 7]]


In [ ]:
# Tokenizer.fit_stransform 테스트
print(tokenizer.fit_transform(sequence))

[[1, 2, 3, 4, 5], [1, 6, 7]]


# 문제 2 ### **TfidfVectorizer 생성하기**

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
$$idf(d,t)=log_e(\frac{n}{1+df(d,t)})$$
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
|  | 토큰1 | 토큰2 | 토큰3 |
| --- | --- | --- | --- |
| 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
| 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |

- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
$$tf-idf(d,t) = tf(d,t) \times idf(d,t)$$

In [ ]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    '''
    문제 2-1.
    '''
    from math import log                                                        # log 계산을 위한 math 라이브러리

    self.idf_matrix = []                                                        # idf matrix 생성
    n = len(sequences)                                                          # 입력된 문장 갯수
    word_dict = self.tokenizer.word_dict                                        # tokenizer가 생성한 단어사전
    df_list = []                                                                # 계산할 df 리스트
    
    for id in list(word_dict.values())[1:]:                                     # 0번째 토큰을 제외한 1번토큰부터 시작
      df = 0                                                                    # df값 초기화
      for s in tokenized:                                                       # 토큰화된 문장들에서 1문장씩 불러옴
        if id in s:                                                             # 토큰(단어)의 id 가 토큰화된 문장에 있다면
          df += 1                                                               # df값 상승
      idf = log(n/(1+df))                                                       # 주어진 공식으로 idf 계산
      self.idf_matrix.append(idf)                                               # idf 매트릭스에 idf값 넣기

    self.fit_checker = True
    

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.tokenizer.transform(sequences)
      '''
      문제 2-2.
      '''
      vocab_id_list = list(self.tokenizer.word_dict.values())[1:]               # oov를 제외한 단어들의 value 값 리스트
      self.tfidf_matrix = []                                                    # 전체 tfidf
      for s in tokenized:
        tf = 0
        tfidf = 0
        tfidf_s = []                                                            # 문장별 tfidf
        for id in vocab_id_list:                                                 
          tf = s.count(id)                                                      # tf 계산 
          tfidf = tf * self.idf_matrix[id-1]                                    # tfidf 계산
          tfidf_s.append(tfidf)                                                 # 문장별 tfidf에 추가
        self.tfidf_matrix.append(tfidf_s)                                       # 전체 tfidf에 추가

      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [ ]:
# TfidfVectorizer.fit 테스트
tfidf = TfidfVectorizer(tokenizer)

tfidf.fit(sequence)
print(tfidf.idf_matrix)

[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# TfidfVectorizer.transform 테스트
tfidf.transform(sequence)

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [ ]:
# TfidfVectorizer.fit_transform 테스트
tfidf.fit_transform(sequence)

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]